In [73]:
# for multiple output on Jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to open the web page for web scraping
import requests

# to parse html 
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
# selenium or splinter for 
from splinter import Browser
import numpy as np


In [2]:
def compile_URL(job,city):
    """
    """
    base_url = 'http://www.indeed.com/jobs?q='
    add_1_job= job
    add_precity = '&l='
    add_2_city=city
    return (base_url + add_1_job + add_precity + add_2_city)


In [4]:
# print(compile_URL('developer','denver'))

In [5]:
def bring_soup(url):
    html_source= requests.get(url)
    soup = BeautifulSoup(html_source.content, 'html.parser', from_encoding="utf-8")
    return soup

In [19]:
soup = bring_soup(compile_URL('developer','denver'))

In [20]:
def title_column(soup):
    col_title =[]
    for itersoup in soup.find_all(class_= "result" ):
        # - - - - 
        try:
            itertitle = itersoup.find(class_='jobtitle').\
            text.replace('\n', '')
        except:
            itertitle='None'
        # - - - - 
        col_title.append(itertitle)
        
    return col_title

In [21]:
# title_column(soup)

In [22]:
def location_column(soup):
    col_location  = [ ]
    for itersoup in soup.find_all(class_= "result" ):
            try:
                location = itersoup.find('span', {'class':"location" }).\
                text.replace('\n', '')
            except:
                location = 'None'
#             - - -
            col_location.append(location)
    return col_location

In [23]:
# location_column(soup)

In [24]:
def company_column(soup):
    col_company  = [ ]
    for itersoup in soup.find_all(class_= "result" ):
            try:
                company = itersoup.find(class_='company').\
                text.replace('\n','')
            except:
                company = 'None'
#             - - -
            col_company.append(company.strip())
    return col_company

In [25]:
# company_column(soup)

In [26]:
def desc_column(soup):
    col_desc  = [ ]
    for itersoup in soup.find_all(class_= "result" ):
            try:
                description = itersoup.find('span', {'class':'summary'}).\
                text.replace('\n', '')
            except:
                description = 'None'
#             - - -
            col_desc.append(description.strip())
    return col_desc

In [ ]:
# desc_column(soup)

In [28]:
def url_column(soup):
    col_href = []
    base_url = 'http://www.indeed.com'
    for iterfoo in soup.find_all(class_= "result" ):
        try:
            href = iterfoo.find('a')['href']
            col_href.append((base_url+href).strip())
        except:
            href='None'
            col_href.append(href)
        
    return list(col_href)
    #     print('http://www.indeed.com'+href)

In [29]:
def frame_indeed(soup):
    """ this function still can not make soup """
    return pd.DataFrame({'Title': title_column(soup),
              'Company': company_column(soup),
              'Location': location_column(soup),
              'Description': desc_column(soup),
              'URLs': url_column(soup)})

In [30]:
# this is the main function of our app's JobLister module
# ====================================
# compile_URL('developer','denver')
def parse_n_frame(url):
#     /*  this function brings soup herself  */
    soup = bring_soup(url)
    return pd.DataFrame({'Title': title_column(soup),
              'Company': company_column(soup),
              'Location': location_column(soup),
              'Description': desc_column(soup),
              'URLs': url_column(soup)})
#============end==============

#===========test==============
# parse_n_frame(dev_denver_url)

In [31]:
def init_browser():
    path_chromedriver = "C:/Users/DENVER/Documents/0000_Attila_Codes_Apps_Hw_Projects/DU_BigData_pr2_ETL_MySQL_Python/chromedriver"
    executable_path = {
        "executable_path":
        "C:/Users/DENVER/Documents/0000_Attila_Codes_Apps_Hw_Projects/DU_BigData_pr2_ETL_MySQL_Python/chromedriver"
    }
    return Browser("chrome", **executable_path, headless=False)
#  - - -
# ----test----
browser = init_browser()

In [32]:
def browse_all_jobs(url):
    browser = init_browser()
    for i in range(1,(len(url_list)-10)):
        browser.visit(url_list[i])

In [33]:
jobListSample = ['data+analyst',
            'developer',
            'programmer',
            'front+end+developer',
            'back+end+developer',
            'full+stack+developer'
           ]

In [34]:
cityListSample = ['denver',
                  'boulder',
             'phoenix',
             'houston',
             'tucson',
             'chicago',
             'new+york',
             'albuquerque',
             'seattle',
             'san+francisco'
             ]

In [35]:
def scrapListFrame(job_list, city_list):
    pass 
    list_of_all_frames =[]
    for job in job_list:
        for city in city_list:
            __url__ = compile_URL(job,city)
            __df__= parse_n_frame(__url__)
            list_of_all_frames.append(__df__)            
    return pd.concat(list_of_all_frames)
# -----------------

In [36]:
# scrapListFrame(jobListSample,cityListSample)
# returns dataframe apprx 1,000 rows

In [37]:
# store above data consisting of concatanated dataframes
concat_df=scrapListFrame(jobListSample,cityListSample)

In [38]:
# len(concat_df)

In [82]:
# write scraped data to csv
file_output_csv01 = str(time.strftime("%m-%d-%y")) +str(np.random.randint(1,9+1)) + "-jobs.csv"
concat_df.to_csv("csv/" + file_output_csv01, encoding="utf-8")

In [45]:
concat_dict = concat_df.to_dict('records')
# converted a dataframe object to list of dicts for mongoDB
# db.insert_many(concat_df.to_dict('records'))

In [70]:
# pandas
out = concat_df.to_json(orient='records')[1:-1].replace('},{', '} {')
# type(out) returns STR

str

In [74]:
with open('file_name.json', 'w') as f:
    f.write(out)

551835